In [7]:
import os
os.chdir('../')

In [584]:
import pandas as pd
from pathlib import Path
from ast import literal_eval
import numpy as np
import wrds
pd.set_option('display.max_columns', None)
from rapidfuzz import fuzz
import re
from source.scrape.get_linkedin_profiles.docs.get_linkedin_profiles import *
from joblib import Parallel, delayed


In [477]:
indir_locations = Path("drive/output/scrape/get_standardized_locations")
indir_profiles = Path('drive/output/scrape/get_linkedin_profiles')
indir_departures = Path('drive/output/derived/contributor_stats/departed_contributors')

In [478]:
df_all_departures = AggregateAllDepartures(indir_departures)

In [84]:
github_profile_locations = pd.read_csv(indir_locations / "standardized_locations_github.csv", index_col = 0)
linkedin_profile_locations = pd.read_csv(indir_locations / "standardized_locations_linkedin.csv", index_col = 0)

departed_committers = pd.read_csv(indir_profiles / 'departed_github_profiles.csv', index_col = 0)
departed_linkedin_profiles = pd.read_parquet(indir_profiles / 'departed_linkedin_profiles.parquet')
departed_linkedin_positions = pd.read_parquet(indir_profiles / 'departed_linkedin_positions.parquet')

In [85]:
departed_committers_clean = departed_committers[['repo_name','actor_id','name_clean','linkedin_url','company','location']]
departed_committers_clean['name_clean'] = departed_committers_clean['name_clean'].replace('[nan]',np.nan)
departed_committers_clean['name_clean'] = departed_committers_clean['name_clean'].apply(lambda x: literal_eval(x) if not pd.isnull(x) else x)

/tmp/user/20506/ipykernel_391209/1377185294.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  departed_committers_clean['name_clean'] = departed_committers_clean['name_clean'].replace('[nan]',np.nan)
/tmp/user/20506/ipykernel_391209/1377185294.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  departed_committers_clean['name_clean'] = departed_committers_clean['name_clean'].apply(lambda x: literal_eval(x) if not pd.isnull(x) else x)


In [86]:
for col in ['standardized_location_address', 'standardized_location_raw']:
    github_profile_locations[col] = github_profile_locations[col].apply(lambda x: literal_eval(x) if not pd.isnull(x) else x)
    linkedin_profile_locations[col] = linkedin_profile_locations[col].apply(lambda x: literal_eval(x) if not pd.isnull(x) else x)

In [109]:
departed_committers_location = pd.merge(departed_committers_clean, github_profile_locations.drop('standardized_location', axis = 1))
departed_committers_location.reset_index(inplace = True)
departed_committers_location['fullname'] = departed_committers_location['name_clean']
departed_committers_location = departed_committers_location.explode('fullname')
departed_profiles_location = pd.merge(departed_linkedin_profiles, linkedin_profile_locations.drop('standardized_location', axis = 1), how = 'left')

In [412]:
departed_profiles_location_merge = departed_profiles_location.rename({'standardized_location_address':'standardized_linkedin_address', 'standardized_location_raw':'raw_linkedin_address'}, axis = 1)\
    [['user_id','fullname','standardized_linkedin_address','raw_linkedin_address','profile_linkedin_url']]

In [413]:
# first, see if there's an exact location match
print("{} unique committers to start with".format(departed_committers_location['index'].unique().shape[0]))
merged_committers = pd.merge(departed_committers_location, departed_profiles_location_merge, on = 'fullname')
print("{} unique committers can be matched to linkedin profiles".format(merged_committers['index'].unique().shape[0]))
merged_committers['location_intersection'] = merged_committers.apply(lambda x: set(x['standardized_location_address']).intersection(x['standardized_linkedin_address']) if type(x['standardized_location_address']) == list
                                                                     and type(x['standardized_linkedin_address']) == list else [], axis = 1)
merged_committers = merged_committers[merged_committers.apply(lambda x: type(x['standardized_location_address']) == list and type(x['standardized_linkedin_address']) == list or
                                                              not pd.isnull(x['company']), axis = 1)]
print("{} unique committers have GH and linkedin locations, or a GH company name".format(merged_committers['index'].unique().shape[0]))
merged_committers['exact_match'] = merged_committers.assign(location_intersection_length = merged_committers['location_intersection'].apply(lambda x: len(x)>0)).groupby(['repo_name','actor_id'])['location_intersection_length']\
    .transform('any')

13502 unique committers to start with
7115 unique committers can be matched to linkedin profiles
5742 unique committers have GH and linkedin locations, or a GH company name


In [414]:
# next, filter out different countries and states
merged_committers['github_country'] = merged_committers['standardized_location_raw'].apply(
    lambda x: set(list([country['address_components'][-1]['long_name'] for country in x])) if type(x) != float else set())
merged_committers['github_state'] = merged_committers['standardized_location_raw'].apply(
    lambda x: set(list([country['address_components'][-2]['long_name'] for country in x if len(country['address_components'])>1])) if type(x) != float else set())
merged_committers['linkedin_country'] = merged_committers['raw_linkedin_address'].apply(
    lambda x: set(list([country['address_components'][-1]['long_name'] for country in x])) if type(x) != float else set())
merged_committers['linkedin_state'] = merged_committers['raw_linkedin_address'].apply(
    lambda x: set(list([country['address_components'][-2]['long_name'] for country in x if len(country['address_components'])>1])) if type(x) != float else set())
for region in ['country','state']:
    merged_committers[f'{region}_overlap'] = merged_committers.apply(lambda x: x[f'github_{region}'].intersection(x[f'linkedin_{region}']), axis = 1)
merged_committers = merged_committers[merged_committers.apply(lambda x: (len(x['country_overlap'])>0 and len(x['state_overlap'])>0) or x['exact_match'] or
                                                              (not pd.isnull(x['company']) and type(x['standardized_location_address']) == float or type(x['standardized_linkedin_address']) == float), axis = 1)]
print("{} unique committers have a potential location match or a GH company name".format(merged_committers['index'].unique().shape[0]))

4396 unique committers have a potential location match or a GH company name


In [415]:
# if a perfect match exists, filter out non perfect matches
merged_committers = merged_committers[merged_committers.apply(lambda x: not (x['exact_match'] and len(x['location_intersection'])==0), axis = 1)]
departed_positions = departed_linkedin_positions[departed_linkedin_positions['user_id'].isin(merged_committers['user_id'].tolist())]

In [416]:
params = dict()
company_list = tuple(departed_positions['rcid'].dropna().drop_duplicates().tolist())
params['company_list'] = company_list

db = wrds.Connection(wrds_username="chrisliao")
company_mappings = db.raw_sql("""SELECT * FROM revelio.company_mapping WHERE rcid in %(company_list)s""", params = params)
db.close()

Loading library list...
Done


In [417]:
departed_company_mappings = pd.merge(departed_positions, company_mappings[['rcid','company','ultimate_parent_company_name']])[['user_id','company','ultimate_parent_company_name']].drop_duplicates()\
    .rename({'company':'linkedin_company_name'}, axis = 1)
company_list = departed_company_mappings.groupby('user_id').agg({'linkedin_company_name':list, 'ultimate_parent_company_name':list}).reset_index()
merged_committers['merge_count'] = merged_committers.groupby(['repo_name','actor_id'])['name_clean'].transform('count')
merged_committers = pd.merge(merged_committers, company_list, how = 'left')
# filter out those we can't match by company (linkedin, github)
merged_committers = merged_committers[merged_committers.apply(lambda x:  not (x['merge_count']>1 and type(x['linkedin_company_name']) == float or pd.isnull(x['company'])), axis = 1)]
print("{} unique committers have a potential location match or a potential GH company match".format(merged_committers['index'].unique().shape[0]))

3154 unique committers have a potential location match or a potential GH company match


In [418]:
merged_committers['linkedin_company_matches'] = merged_committers.apply(
    lambda x: [(comp_name, round(fuzz.ratio(x['company'].lower(), comp_name.lower()), 2)) for comp_name in x['linkedin_company_name']] 
    if type(x['linkedin_company_name']) == list and not pd.isnull(x['company']) else np.nan, axis = 1)

In [419]:
merged_committers['company_match'] = merged_committers['linkedin_company_matches'].apply(lambda x: any([score[1]>45 for score in x]) if type(x) == list else False)
merged_committers['max_score'] = merged_committers['linkedin_company_matches'].apply(lambda x: max([score[1] for score in x]) if type(x) == list else np.nan)
merged_committers = merged_committers.query('company_match | max_score.isna()')
merged_committers['merge_count'] = merged_committers.groupby(['repo_name','actor_id'])['name_clean'].transform('count')
merged_committers = pd.merge(merged_committers, departed_linkedin_profiles[['user_id','updated_dt']])
print("{} unique committers have a potential location match & a potential GH company match".format(merged_committers['index'].unique().shape[0]))

1975 unique committers have a potential location match & a potential GH company match


In [420]:
merged_committers = merged_committers.sort_values(['max_score','updated_dt'], ascending = False).drop_duplicates(['repo_name','actor_id'])
merged_committers = merged_committers.drop(['name_clean','linkedin_url','index'], axis = 1)

In [428]:
departed_committers_linkedin = pd.merge(departed_committers_clean, merged_committers, how = 'left')[['repo_name','actor_id','linkedin_url','profile_linkedin_url','user_id']]
departed_committers_linkedin['linkedin_url'] = departed_committers_linkedin.apply(
    lambda x: x['linkedin_url'] if not pd.isnull(x['linkedin_url']) else x['profile_linkedin_url'], axis = 1)
departed_committers_linkedin.drop('profile_linkedin_url', axis = 1, inplace = True)

In [443]:
pattern = r'^(?:https?:\/\/)?(?:www\.)?linkedin\.com/(.*)$'
departed_committers_linkedin['linkedin_url'] = departed_committers_linkedin['linkedin_url'].apply(lambda x: re.sub(pattern, r'linkedin.com/\1', x) if not pd.isnull(x) else x)

In [448]:
params = dict()
linkedin_profiles = tuple(departed_committers_linkedin.query('~linkedin_url.isna() & user_id.isna()')['linkedin_url'].tolist())
params['linkedin_profiles'] = linkedin_profiles

db = wrds.Connection(wrds_username="chrisliao")
github_linkedin_profiles = db.raw_sql("""SELECT * FROM revelio.individual_user WHERE profile_linkedin_url in %(linkedin_profiles)s""", params = params)
db.close()

Loading library list...
Done


In [461]:
profile_user_dict = github_linkedin_profiles[['profile_linkedin_url','user_id']].set_index('profile_linkedin_url').to_dict()['user_id']
departed_committers_linkedin['user_id'] = departed_committers_linkedin.apply(lambda x: profile_user_dict.get(x['linkedin_url'], x['user_id']), axis = 1)

In [475]:
db = wrds.Connection(wrds_username="chrisliao")
departed_linkedin_profiles_all = QueryForLinkedinPositions(departed_committers_linkedin.dropna(), db)

Loading library list...
Done


In [531]:
# 1. get positions for relevant users
# 2. see if in the hypothesized departure period, they changed jobs
#departed_linkedin_profiles_all.groupby('user_id')['
for datecol in ['startdate','enddate']:
    departed_linkedin_profiles_all[datecol] = pd.to_datetime(departed_linkedin_profiles_all[datecol])
departed_job_change_dates = departed_linkedin_profiles_all.groupby('user_id').agg({'startdate':list, 'enddate':list}).reset_index()
for datecol in ['startdate','enddate']:
    departed_job_change_dates[datecol] = departed_job_change_dates[datecol].apply(lambda x: list(set([ele.date() for ele in x if not pd.isnull(ele)])))

In [535]:
departed_full = pd.merge(departed_committers_linkedin, departed_job_change_dates, how = 'left')
for datecol in ['startdate','enddate']:
    departed_full[datecol] = departed_full[datecol].apply(lambda x: np.nan if type(x) == list and len(x) == 0 else x)

In [607]:
def SummarizeSpecification(df_specification, idx, time_period, rolling_window, departed_full, indir_departures):
    criteria_pct = df_specifications.loc[idx,'criteria_pct']
    consecutive_periods = df_specifications.loc[idx,'consecutive_periods']
    post_period_length = df_specifications.loc[idx,'post_period_length']
    decline_type = df_specifications.loc[idx,'decline_type']
    decline_stat = df_specifications.loc[idx,'decline_stat']
    if decline_stat == 0 or decline_type == "threshold_gap_qty":
        decline_stat = int(decline_stat)
    
    df_departed = pd.read_parquet(indir_departures / f'departed_contributors_major_months{time_period}_window{rolling_window}D_criteria_commits_{criteria_pct}pct_consecutive{consecutive_periods}_post_period{post_period_length}_{decline_type}_{decline_stat}.parquet')
    df_departure_range = CleanDepartures(df_departed, decline_type)
    df_departure_range = pd.merge(df_departure_range, departed_full, how = 'left')
    df_departure_range['startdate_dep'] = df_departure_range.apply(
        lambda x: any([x['time_range'][0] <= ele <= x['time_range'][1] for ele in x['startdate']]) if type(x['startdate']) == list else False, axis = 1)
    df_departure_range['enddate_dep'] = df_departure_range.apply(
        lambda x: any([x['time_range'][0] <= ele <= x['time_range'][1] for ele in x['enddate']]) if type(x['enddate']) == list else False, axis = 1)
    df_departure_range['corresponding_departure'] = df_departure_range.apply(
        lambda x: x['startdate_dep'] or x['enddate_dep'], axis = 1)
    departure_count = df_departure_range.shape[0]
    linkedin_match_pct = 1-df_departure_range['user_id'].isna().mean()
    departure_occurred_pct = df_departure_range.query('~user_id.isna()')['corresponding_departure'].mean()
    
    return [time_period, rolling_window, 'commits', criteria_pct, consecutive_periods, post_period_length, decline_type, decline_stat,\
            departure_count, linkedin_match_pct, departure_occurred_pct]

def CleanDepartures(df_departed, decline_type):
    if decline_type == "threshold_gap_qty":
        df_departed = df_departed.query('below_qty_mean_gap0 == 1 | below_qty_mean_gap1 == 1')

    df_departed = pd.merge(df_departed, df_departed.query('time_period == final_period')\
                           [['repo_name','actor_id','grouped_index']].rename({'grouped_index':'final_index'}, axis = 1))
    if decline_type == "threshold_gap_qty":
        df_departed['final_index'] = df_departed.apply(
            lambda x: x['final_index'] if x['below_qty_mean_gap0'] == 1 else x['final_index']+1, axis = 1)
    df_departed = pd.merge(df_departed.drop('final_period', axis = 1), df_departed.query('grouped_index == final_index')[['actor_id','repo_name','time_period']]\
                          .rename({'time_period':'final_period'}, axis = 1))
    df_departed['first_post_period_index'] = df_departed['final_index'] + 1
    df_departed['relative_time'] = (df_departed['grouped_index'] - df_departed['final_index'])-1

        
    df_departed['time_period'] = pd.to_datetime(df_departed['time_period'])
    df_departure_range = df_departed.query('relative_time == -1 | relative_time == 0')\
        .groupby(['repo_name','actor_id']).agg({'time_period':list}).reset_index()
    df_departure_range['time_range'] = df_departure_range['time_period'].apply(lambda x: [x[0].date(), x[1].date()])

    return df_departure_range.drop('time_period', axis = 1)

In [630]:
pd.read_parquet('drive/output/derived/project_outcomes/abandoned_projects/abandoned_projects_consecutive_req2_permanentTrue.parquet')
df_departed

,repo_name,abandoned_date
9,007gzs/django_restframework_apiview,2021-07-01
14,0compute/yanc,2015-07-01
34,1e0ng/simhash,2022-07-01
45,4Catalyzer/flask-resty-tenants,2020-07-01
58,ARMmbed/mbed-cloud-sdk-python,2021-01-01
...,...,...
23556,zerok/celery-prometheus-exporter,2020-01-01
23573,zertrin/clikraken,2021-07-01
23588,zopefoundation/Products.ZCTextIndex,2020-07-01
23596,zyga/guacamole,2016-01-01


In [609]:
%%time
departure_validation_cols = ['time_period','rolling_window','criteria_col','criteria_pct','consecutive_periods',
                             'post_period_length','decline_type','decline_stat', 'departure_count', 'linkedin_match_pct', 'departure_occurred_pct']
df_departure_linkedin_validation = pd.DataFrame(columns = departure_validation_cols)
for time_period in [2,3,6]:
    for rolling_window in [732, 1828]:
        print(time_period, rolling_window)
        df_specifications = pd.read_csv(indir_departures / f'departed_contributors_specification_summary_major_months{time_period}_window{rolling_window}D.csv').query('criteria_col == "commits"')
        results = [SummarizeSpecification(df_specifications, idx, time_period, rolling_window, departed_full, indir_departures) for idx in df_specifications.index]
        tmp_df = pd.DataFrame(results, columns=departure_validation_cols)
        df_departure_linkedin_validation = pd.concat(
            [df_departure_linkedin_validation, tmp_df],
            ignore_index=True
        )

2 732


<timed exec>:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


2 1828
3 732
3 1828
6 732
6 1828
CPU times: user 1min 2s, sys: 3.47 s, total: 1min 5s
Wall time: 57.8 s


In [620]:
# have to exclude abandoned projects
df_departure_linkedin_validation

,time_period,rolling_window,criteria_col,criteria_pct,consecutive_periods,post_period_length,decline_type,decline_stat,departure_count,linkedin_match_pct,departure_occurred_pct
0,2,732,commits,75,3,3,threshold_mean,0.2,2523,0.195402,0.176471
1,2,732,commits,75,3,3,threshold_mean,0.1,1545,0.194175,0.206667
2,2,732,commits,75,3,3,threshold_mean,0.0,458,0.194323,0.179775
3,2,732,commits,75,3,3,threshold_gap_qty,0.0,1536,0.147786,0.215859
4,2,732,commits,75,3,3,threshold_gap_qty,5.0,2458,0.164768,0.187654
...,...,...,...,...,...,...,...,...,...,...,...
395,6,1828,commits,90,6,4,threshold_mean,0.2,300,0.203333,0.360656
396,6,1828,commits,90,6,4,threshold_mean,0.1,206,0.228155,0.382979
397,6,1828,commits,90,6,4,threshold_mean,0.0,28,0.107143,0.666667
398,6,1828,commits,90,6,4,threshold_gap_qty,0.0,87,0.172414,0.333333


NameError: name 'asd' is not defined